In [1]:
import pandas as pd
from torch.utils.data import DataLoader,Dataset
import torch
import torch.nn as nn

In [2]:
data = pd.read_csv('./train.csv').iloc[:, 1:-1]
data_test = pd.read_csv('./train.csv').iloc[:, 1:-1]
data.drop(columns=['Episode_Title'], inplace=True)
data_test.drop(columns=['Episode_Title'], inplace=True)
data.shape



(750000, 9)

In [3]:
data['Episode_Length_missing'] = data['Episode_Length_minutes'].isna().astype(int)


In [4]:
data.shape


(750000, 10)

In [5]:
data

,Podcast_Name,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Length_missing
0,Mystery Matters,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,1
1,Joke Junction,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,0
2,Study Sessions,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,0
3,Digital Digest,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,0
4,Mind & Body,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,0
...,...,...,...,...,...,...,...,...,...,...
749995,Learning Lab,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative,0
749996,Business Briefs,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral,0
749997,Lifestyle Lounge,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative,0
749998,Style Guide,108.98,Lifestyle,45.39,Thursday,Morning,93.27,0.0,Negative,0


In [160]:
data_test.shape

(750000, 9)

In [161]:
y_train = data[['Episode_Length_minutes']]
y_train.shape

(539048, 1)

Episode_Length_minutes    64.556952
dtype: float64

In [162]:
y_train.head(10)

,Episode_Length_minutes
1,1.679572
2,0.284060
3,0.079445
4,1.397125
6,0.160318
7,-0.487576
10,1.272471
11,1.298314
12,0.534583
13,0.849561


In [163]:
x_train = data[['Podcast_Name','Genre','Host_Popularity_percentage','Publication_Day','Publication_Time','Number_of_Ads','Episode_Sentiment']]
x_train_test = data_test[['Podcast_Name','Genre','Host_Popularity_percentage','Publication_Day','Publication_Time','Number_of_Ads','Episode_Sentiment']]


In [164]:
x_train.head(10)

,Podcast_Name,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Number_of_Ads,Episode_Sentiment
1,Joke Junction,Comedy,66.95,Saturday,Afternoon,2.0,Negative
2,Study Sessions,Education,69.97,Tuesday,Evening,0.0,Negative
3,Digital Digest,Technology,57.22,Monday,Morning,2.0,Positive
4,Mind & Body,Health,80.07,Monday,Afternoon,3.0,Neutral
6,Criminal Minds,True Crime,35.82,Sunday,Night,0.0,Neutral
7,News Roundup,News,44.99,Thursday,Night,0.0,Positive
10,Sports Central,Sports,84.29,Thursday,Evening,3.0,Positive
11,Melody Mix,Music,78.12,Monday,Evening,1.0,Positive
12,Game Day,Sports,86.65,Saturday,Morning,1.0,Neutral
13,Gadget Geek,Technology,68.42,Wednesday,Afternoon,0.0,Neutral


In [165]:
x_train.dtypes

Podcast_Name                   object
Genre                          object
Host_Popularity_percentage    float64
Publication_Day                object
Publication_Time               object
Number_of_Ads                 float64
Episode_Sentiment              object
dtype: object

In [166]:
x_train = pd.get_dummies(x_train).astype(float)
x_train_test = pd.get_dummies(x_train_test).astype(float)
x_train.shape

(539048, 74)

In [167]:
from torch.utils.data import Dataset
import torch
class MyDataset(Dataset):
    def __init__(self, data,mode, target=None):
        self.mode = mode
        if mode == "test":
            self.data = torch.tensor(data).float()
        elif mode == "train":
            self.data = torch.tensor(data).float()
            self.target = torch.tensor(target).float()
    def __getitem__(self, index):
        if self.mode == "test":
            return self.data[index]
        return self.data[index],self.target[index]
    def __len__(self):
        return len(self.data)

In [168]:
train_loader = DataLoader(MyDataset(x_train.values,mode = "train",target=y_train.values), batch_size=1024, shuffle=True)
test_loader = DataLoader(MyDataset(x_train_test.values,mode="test"), batch_size=1024, shuffle=True)

In [169]:
# class model(torch.nn.Module):
#     def __init__(self,input_size,output_size):
#         super(model, self).__init__()
#         self.nn = nn.Sequential(
#             nn.Linear(input_size,256),
#             nn.BatchNorm1d(256),
#             nn.Dropout(0.2),
#             nn.ReLU(),
#             nn.Linear(256,output_size),
#
#         )
#     def forward(self, x):
#         return self.nn(x)
class model(nn.Module):
    def __init__(self, input_size, output_size):
        super(model, self).__init__()
        self.nn = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),  # 改用LeakyReLU防止神经元死亡
            nn.Dropout(0.3),     # 提高Dropout率

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),

            nn.Linear(128, output_size),
            # 如果目标值在[0,1]或[0,100]范围内可添加：
            # nn.Sigmoid()  # 或 nn.Tanh()
        )

    def forward(self, x):
        return self.nn(x)


In [170]:
epochs = 20
loss_fn = nn.MSELoss()
model = model(input_size=x_train.shape[1],output_size=y_train.shape[1]).to(torch.device('cuda:0'))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(epochs):
    loss_all = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data,target = data.to(torch.device('cuda:0')),target.to(torch.device('cuda:0'))
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss_all += loss.item()
        loss.backward()
        optimizer.step()
    print(f"epoch {epoch+1}, loss {loss_all/len(train_loader)}")




epoch 1, loss 1.0068868778688405
epoch 2, loss 0.9937074409490292
epoch 3, loss 0.9934096101113017
epoch 4, loss 0.9925611018003504
epoch 5, loss 0.9924692527392783
epoch 6, loss 0.9922678109816854
epoch 7, loss 0.9920480105410944
epoch 8, loss 0.9918430998384161
epoch 9, loss 0.9915680382011963
epoch 10, loss 0.9912784040765925
epoch 11, loss 0.9911779713811639
epoch 12, loss 0.9909265603467229
epoch 13, loss 0.9908365522435313
epoch 14, loss 0.990428936549337
epoch 15, loss 0.9903366822445415
epoch 16, loss 0.9902144188672813
epoch 17, loss 0.9901841918476618
epoch 18, loss 0.9899953611661633
epoch 19, loss 0.9897827823881407
epoch 20, loss 0.9897639565947386


In [171]:
outputs=[]
with torch.no_grad():
    model.eval()
    for batch_idx, (data) in enumerate(test_loader):
        data = data.to(torch.device('cuda:0'))
        output = model(data)
        outputs.append(output)

In [210]:
outputs_tensor = torch.cat(outputs, dim=0).cpu().detach().numpy()


In [211]:
df_results = pd.DataFrame(outputs_tensor, columns=['Episode_Length_minutes'])
df_results.to_csv("guest.csv", index=False)
print("预测结果已保存为 guest.csv")

预测结果已保存为 guest.csv
